<img src="https://drive.google.com/uc?export=view&id=1wYSMgJtARFdvTt5g7E20mE4NmwUFUuog" width="200">

[![Build Fast with AI](https://img.shields.io/badge/BuildFastWithAI-GenAI%20Bootcamp-blue?style=for-the-badge&logo=artificial-intelligence)](https://www.buildfastwithai.com/genai-course)
[![EduChain GitHub](https://img.shields.io/github/stars/satvik314/educhain?style=for-the-badge&logo=github&color=gold)](https://github.com/satvik314/educhain)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/16UjTf2r6BtlT5B-La40UqJN4Ep17dAPu?usp=sharing)
## Master Generative AI in 6 Weeks
**What You'll Learn:**
- Build with Latest LLMs
- Create Custom AI Apps
- Learn from Industry Experts
- Join Innovation Community
Transform your AI ideas into reality through hands-on projects and expert mentorship.
[Start Your Journey](https://www.buildfastwithai.com/genai-course)
*Empowering the Next Generation of AI Innovators

# 🚀 Agenta: Open-Source LLMOps Platform

[Agenta](https://github.com/Agenta-AI/agenta) simplifies **LLMOps** with tools for **development, deployment, and monitoring** of LLM-powered apps. It supports **RAG, agents,** and frameworks like **LlamaIndex & LangChain**.

## 🌟 Key Features

- 🎭 **Prompt Playground** – Test & compare 50+ LLMs.  
- 🔄 **Custom Workflows** – Build RAG & agent-based systems.  
- 📊 **LLM Evaluation** – Use built-in & custom evaluators.  
- 🧑‍⚖️ **Human Evaluation** – A/B testing & expert annotations.  
- 🔍 **Prompt Management** – Version control & configurations.  
- 👀 **Observability & Tracing** – Debug & monitor in real time.

### 📦 **Dependency Installation**  








In [ ]:
pip install -U agenta openai opentelemetry-instrumentation-openai

In [ ]:
pip install opentelemetry-instrumentation-langchain langchain_community langchain_openai instructor litellm

### **🔑 Setup API Keys**








In [ ]:

import os
from google.colab import userdata

os.environ['OPENAI_API_KEY']=userdata.get('OPENAI_API_KEY')

os.environ["AGENTA_API_KEY"] = userdata.get("AGENTA_API_KEY")
os.environ["AGENTA_HOST"] = "https://cloud.agenta.ai"

### **🤖 Initialize Agenta & OpenAI Client**








In [ ]:
import agenta as ag
from opentelemetry.instrumentation.openai import OpenAIInstrumentor
from openai import OpenAI
client = OpenAI()

ag.init()

OpenAIInstrumentor().instrument()


response= client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Write a short story about AI Engineering."},
    ],
)

print(response.choices[0].message.content)

### **🛠️ Instrumenting a Workflow with a Parent Span**








In [ ]:
import nest_asyncio
nest_asyncio.apply()

import agenta as ag
from opentelemetry.instrumentation.openai import OpenAIInstrumentor

from openai import OpenAI
import asyncio

client = OpenAI()

ag.init()
OpenAIInstrumentor().instrument()


@ag.instrument(spankind="TASK")
async def generate_story_prompt(topic: str, genre: str):
    return f"Write a {genre} story about {topic}."


@ag.instrument(spankind="WORKFLOW")
async def generate_story(topic: str, genre: str):
    prompt = await generate_story_prompt(topic, genre)
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "user",
                "content": prompt,
            },
        ],
    )
    return response.choices[0].message.content


async def main():
    result = await generate_story(topic="the future", genre="sci-fi")
    print(result)

if __name__ == "__main__":
    try:
        asyncio.run(main())
    except RuntimeError as e:
        if "cannot be called from a running event loop" in str(e):
            loop = asyncio.get_event_loop()
            if loop.is_running():
                print("An event loop is already running.  Attempting to use it.")
                loop.run_until_complete(main())
            else:
                print("No running event loop found, even though RuntimeError was raised.")
                raise
        else:
            raise

### **🔗 Instrumenting a Workflow with LangChain**








In [ ]:
import agenta as ag
from opentelemetry.instrumentation.langchain import LangchainInstrumentor
from langchain.schema import SystemMessage, HumanMessage
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_community.chat_models import ChatOpenAI
from langchain.chains import LLMChain, SequentialChain, TransformChain

ag.init()
LangchainInstrumentor().instrument()


def langchain_app():
    chat = ChatOpenAI(temperature=0)

    transform = TransformChain(
        input_variables=["subject"],
        output_variables=["prompt"],
        transform=lambda inputs: {"prompt": f"Tell me a joke about {inputs['subject']}."},
    )

    first_prompt_messages = [
        SystemMessage(content="You are a funny sarcastic nerd."),
        HumanMessage(content="{prompt}"),
    ]
    first_prompt_template = ChatPromptTemplate.from_messages(first_prompt_messages)
    first_chain = LLMChain(llm=chat, prompt=first_prompt_template, output_key="joke")

    second_prompt_messages = [
        SystemMessage(content="You are an Elf."),
        HumanMessagePromptTemplate.from_template(
            "Translate the joke below into Sindarin language:\n{joke}"
        ),
    ]
    second_prompt_template = ChatPromptTemplate.from_messages(second_prompt_messages)
    second_chain = LLMChain(llm=chat, prompt=second_prompt_template)

    workflow = SequentialChain(
        chains=[transform, first_chain, second_chain],
        input_variables=["subject"],
    )

    result = workflow({"subject": "OpenTelemetry"})
    print(result)

langchain_app()

### **📚 Instrumenting a Workflow with Instructor**








In [ ]:

import agenta as ag
import openai
import instructor
from pydantic import BaseModel
from opentelemetry.instrumentation.openai import OpenAIInstrumentor

ag.init()

OpenAIInstrumentor().instrument()

class UserInfo(BaseModel):
    name: str
    age: int

@ag.instrument(spankind="WORKFLOW")
def instructor_workflow():
    client = instructor.from_openai(openai.OpenAI())

    user_info = client.chat.completions.create(
        model="gpt-3.5-turbo",
        response_model=UserInfo,
        messages=[{"role": "user", "content": "John Doe is 30 years old."}],
    )
    return user_info

user_info = instructor_workflow()
print(user_info)

### **⚡ Instrumenting a Workflow with LiteLLM**








In [ ]:
import nest_asyncio
nest_asyncio.apply()

import agenta as ag
import litellm
import asyncio

ag.init()

@ag.instrument()
async def agenerate_completion():
    litellm.callbacks = [ag.callbacks.litellm_handler()]

    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Write a short story about AI Engineering."},
    ]
    temperature = 0.2
    max_tokens = 100

    chat_completion = await litellm.acompletion(
        model="gpt-3.5-turbo",
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens,
    )
    print(chat_completion)


async def main():
    await agenerate_completion()


asyncio.get_event_loop().run_until_complete(main())